# Lab 2 with DeepSeek Integration

This notebook demonstrates how to use DeepSeek as an alternative to Anthropic in Lab 2. DeepSeek offers a cost-effective alternative that uses the OpenAI-compatible API format.

## Author
**Moiz M.** - *October 2025*

## Key Features
- Uses DeepSeek's chat model
- Demonstrates OpenAI-compatible API usage
- Shows how to integrate alternative LLM providers
- Cost-effective alternative to other providers

## Setup Requirements
1. DeepSeek API key (get one from https://platform.deepseek.com/)
2. Add `DEEPSEEK_API_KEY` to your `.env` file


In [ ]:
# Start with imports
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

# Load environment variables
load_dotenv(override=True)

# Set up DeepSeek configuration
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")

# Verify API key
if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins with: {deepseek_api_key[:8]}")
else:
    print("DeepSeek API Key not found in environment variables")


In [ ]:
# Generate a challenging question
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

# Create DeepSeek client
deepseek = OpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)

# Get the question from DeepSeek
response = deepseek.chat.completions.create(
    model="deepseek-chat",
    messages=messages
)
question = response.choices[0].message.content
print("Question:", question)


In [ ]:
# Initialize lists to store results
competitors = []
answers = []

# Set up the question for all competitors
messages = [{"role": "user", "content": question}]

# Get answers from different models
models = {
    "deepseek-chat": (deepseek, "deepseek-chat"),
    "gpt4-mini": (OpenAI(), "gpt-4o-mini"),
    "local-llama": (OpenAI(base_url='http://localhost:11434/v1', api_key='ollama'), "llama3.2")
}

for model_name, (client, model) in models.items():
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        answer = response.choices[0].message.content
        
        print(f"\nResponse from {model_name}:")
        display(Markdown(answer))
        
        competitors.append(model_name)
        answers.append(answer)
    except Exception as e:
        print(f"Error with {model_name}: {str(e)}")


In [ ]:
# Create judging prompt
together = ""
for index, answer in enumerate(answers):
    together += f"# Response from competitor {index+1}\n\n"
    together += answer + "\n\n"

judge = f"""You are judging a competition between {len(competitors)} competitors.
Each model has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else."""

# Get judgment from DeepSeek
judge_messages = [{"role": "user", "content": judge}]
response = deepseek.chat.completions.create(
    model="deepseek-chat",
    messages=judge_messages
)
results = response.choices[0].message.content

# Display results
try:
    results_dict = json.loads(results)
    ranks = results_dict["results"]
    print("\nFinal Rankings:")
    for index, result in enumerate(ranks):
        competitor = competitors[int(result)-1]
        print(f"Rank {index+1}: {competitor}")
except Exception as e:
    print(f"Error parsing results: {str(e)}")
    print("Raw results:", results)


## Improvements and Differences from Original Lab 2

1. **Simplified Setup**: Uses OpenAI-compatible API format for all providers
2. **Error Handling**: Added try-catch blocks to handle potential API failures gracefully
3. **Cost Efficiency**: Uses DeepSeek as the main model, which is generally more cost-effective
4. **Streamlined Code**: Combined model calls into a single loop with consistent interface
5. **Local Option**: Includes Ollama integration for those who want to run models locally

## Usage Notes

1. Make sure to have your DeepSeek API key in the `.env` file
2. For the local Llama model, ensure Ollama is installed and running
3. The code will gracefully handle missing API keys or failed API calls
4. You can easily add more models by extending the `models` dictionary

## Cost Considerations

DeepSeek typically offers more competitive pricing compared to other providers:
- Lower cost per token
- Minimum deposit of only $2
- Good performance for most tasks

## Contributing

Feel free to extend this example with:
- Additional model providers
- Different evaluation criteria
- Enhanced error handling
- UI improvements
